In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

window_length = 40
window_step = 3
number_of_epochs = 100
generated_text_length = 1000
batch_size = 100
test_input_file = 'data/holmes/test-holmes.txt'
input_file = 'data/holmes/holmes.txt'
output_file =  'holmes-by-char.txt'

file = open(input_file, 'r') 
text = file.read()
file.close()
#text = text.lower()
# replace newlines with blanks, and double blanks with singles
text = text.replace('\n',' ') 
text = text.replace('  ', ' ')
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# make overlapping sequences of window_length characters
sentences = []
next_chars = []
for i in range(0, len(text)-window_length, window_step):
    sentences.append(text[i: i + window_length])
    next_chars.append(text[i + window_length])
print('number of sequences of ',window_length,':', len(sentences))

# Turn inputs and targets into one-hot versions
X = np.zeros((len(sentences), window_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# build the model
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(window_length, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def print_string(out_str=''):
    print(out_str, end='')
    File_writer.write(out_str)
    
File_writer = open(output_file, 'w')

# train the model, output generated text after each iteration
for iteration in range(number_of_epochs):
    print_string('--------------------------------------------------\n')
    print_string('Iteration '+str(iteration)+'\n')
    history = model.fit(X, y, batch_size=batch_size, epochs=1)
    print_string('  Loss from iteration '+str(iteration)+' = '+str(history.history['loss'])+'\n')
    start_index = random.randint(0, len(text) - window_length - 1)

    for diversity in [0.5, 1.0, 1.5, 2.0]:
        print_string('\n----- diversity: '+str(diversity)+'\n')

        generated = ''
        sentence = text[start_index: start_index + window_length]
        generated += sentence
        print_string('----- Generating with seed: <'+sentence+'>\n')
        
        for i in range(generated_text_length):
            x = np.zeros((1, window_length, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            #print_string(next_char)
            #sys.stdout.write(next_char)
            #sys.stdout.flush()
        print_string(generated+'\n\n')
        #print_string('\n\n')
        File_writer.flush()
print_string('\n')
File_writer.close()

Using Theano backend.


corpus length: 1637265
total chars: 89
number of sequences of  40 : 545742
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 128)           111616    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 89)                11481     
Total params: 254,681
Trainable params: 254,681
Non-trainable params: 0
_________________________________________________________________
--------------------------------------------------
Iteration 0
Epoch 1/1
  3900/545742 [..............................] - ETA: 1546s - loss: 3.3388